In [182]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [183]:
def create_data(path):
    data=pd.read_csv(path)
    data['mean'] = data.mean(axis = 1)
    data['hour'] = data.index // 60
    data['day'] = data['hour'] // 24
    data['week'] = data['day'] // 7
    data['12_hour'] = data['hour'] // 12
    data['day_time'] = data['12_hour'] % 2
    data['4_day_period'] = data['day'] % 4
    return data

In [184]:
def combine(fe, ma):
    return pd.merge(fe, ma[[f'm{i}' for i in range(1, 14)]+['mean']], on=fe.index)\
    .rename(columns={'key_0':'minute', 'mean_x':'fe_mean', 'mean_y':'ma_mean'})

In [185]:
fe_temp = create_data('data/fem_temp.csv')
ma_temp = create_data('data/male_temp.csv')

In [186]:
temp = combine(fe_temp, ma_temp)

In [187]:
fe_act = create_data('data/fem_act.csv')
ma_act = create_data('data/male_act.csv')

In [188]:
act = combine(fe_act, ma_act)

In [199]:
fe_temp.to_csv('cleaned_data/fem_temp.csv', index=False)
ma_temp.to_csv('cleaned_data/male_temp.csv', index=False)
temp.to_csv('cleaned_data/all_temp.csv', index=False)
fe_act.to_csv('cleaned_data/fem_act.csv', index=False)
ma_act.to_csv('cleaned_data/male_act.csv', index=False)
act.to_csv('cleaned_data/all_act.csv', index=False)

In [189]:
def create_line(df_c, y, day_num=False, title=''):
    if str(day_num)=='0' or day_num:
        df=df_c[df_c['day']==day_num].groupby('hour').mean().reset_index()
        df['hour'] = df['hour'] % 24
        x='hour'
    else:
        df = df_c.groupby('day').mean().reset_index()
        x='day'
    female_ids = [f'f{i}' for i in range(1, 14)]
    male_ids = [f'm{i}' for i in range(1, 14)]

    fig = go.Figure()

    for i, fid in enumerate(female_ids):
        fig.add_trace(go.Scatter(
            x=df[x],
            y=df[fid],
            name="female" if i == 0 else "",           # Only the first gets the legend label
            legendgroup="female",                      # Group all as 'female'
            showlegend=(i == 0),                       # Only the first trace shows in legend
            mode='lines',
            line=dict(color='rgb(255, 210, 210)')
        ))

    for i, mid in enumerate(male_ids):
        fig.add_trace(go.Scatter(
            x=df[x],
            y=df[mid],
            name="male" if i == 0 else "",           
            legendgroup="male",                     
            showlegend=(i == 0),                       
            mode='lines',
            line=dict(color='rgb(210, 210, 255)')
        ))

    fig.add_trace(go.Scatter(
            x=df[x],
            y=df['fe_mean'],
            name="female mean",           
            legendgroup="female mean",                                   
            mode='lines',
            line=dict(color='rgb(255, 0, 0)')
        ))

    fig.add_trace(go.Scatter(
            x=df[x],
            y=df['ma_mean'],
            name="male mean",           
            legendgroup="male mean",                                   
            mode='lines',
            line=dict(color='rgb(0, 0, 255)')
        ))

    fig.update_layout(
        title=title,
        xaxis_title=x,
        yaxis_title=y
    )
    return fig

In [190]:
f=create_line(temp, 'Temperature', day_num=0, title='Body temperature in day 1')
f.show()

In [191]:
f_all=create_line(temp, 'Temperature', title='Body temperature')
f_all.show()

In [192]:
male_df = pd.DataFrame({
    'temperature': ma_temp['mean'],
    'activity': ma_act['mean'],
    'sex': 'male'
})

female_df = pd.DataFrame({
    'temperature': fe_temp['mean'],
    'activity': fe_act['mean'],
    'sex': 'female'
})

plot_df = pd.concat([male_df, female_df], ignore_index=True)
fig = px.scatter(
    plot_df,
    x='temperature',
    y='activity',
    color='sex',  # Automatically assigns different colors to 'male' and 'female'
    color_discrete_map={'male': 'rgb(150, 150, 255)', 'female': 'rgb(255, 150, 150)'},  # Optional: custom colors
    labels={'temperature': 'Core Body Temperature', 'activity': 'Activity Level'}
)
fig.update_layout(
    title='core body temperature vs. activity level'
)
fig.show()

In [193]:
def get_alpha_colors(values, base_rgb):
    # Normalize values to [0.2, 1] for visibility; adjust as needed
    norm = (values - np.min(values)) / (np.max(values) - np.min(values))
    alphas = 0.2 + 0.8 * norm
    return [f'rgba({base_rgb},{a:.2f})' for a in alphas]


In [194]:
fig = go.Figure()
m_4day = ma_temp.groupby('4_day_period').mean().reset_index()
f_4day = fe_temp.groupby('4_day_period').mean().reset_index()
male_colors = get_alpha_colors(m_4day['mean'].values, "0,0,255") 
female_colors = get_alpha_colors(f_4day['mean'].values, "255,0,0")
fig.add_trace(go.Bar(
    x=m_4day['4_day_period'],
    y=m_4day['mean'],
    name='Male',
    marker_color=male_colors
))

fig.add_trace(go.Bar(
    x=f_4day['4_day_period'],
    y=f_4day['mean'],
    name='Female',
    marker_color=female_colors
))

fig.update_layout(
    barmode='group',  # Side-by-side bars
    xaxis_title='Day in 4-Day Cycle',
    yaxis_title='Body Temperature (°C)',
    title='Male vs Female Body Temperature Across 4-Day Cycle'
)

fig.show()

In [195]:
px.bar(data.assign(ith_day = data['4_day_period'].apply(lambda x:str(x+1))).groupby('ith_day')[['mean']].mean().reset_index(),
        x='ith_day', y='mean', color='mean',
       labels={'mean':'mean body temperature', 'ith_day':'i-th day in the 4 day period'})

In [196]:
px.line(data.groupby('day').mean().reset_index(),
        x='day', y=[f'f{i}' for i in range(1, 14)])

In [197]:
px.line(data[data['day']==0].groupby('hour').mean().reset_index(), x='hour', y=[f'f{i}' for i in range(1, 14)])

In [198]:
px.line(data[data['day']==0].groupby('hour').mean().reset_index(), x='hour', y='mean', color='day_time')